In [1]:
import pandas as pd

In [3]:
data=pd.read_csv('diamonds (1).csv')
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


## seggregating the features and target

In [5]:
x=data.drop(columns={'price'})
y=data['price']

## splitting train and test

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [8]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((40455, 9), (13485, 9), (40455,), (13485,))

##### scaling and encoding

In [ ]:
 Features 

	price : price in US dollars (\$326--\$18,823)

	carat : weight of the diamond (0.2--5.01)

	cut   : quality of the cut (Fair, Good, Very Good, Premium, Ideal)

	color : diamond colour, from J (worst) to D (best)

	clarity : a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

	x : length in mm (0--10.74)

	y : width in mm (0--58.9)

	z : depth in mm (0--31.8)

	depth : total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

	table : width of top of diamond relative to widest point (43--95)


In [ ]:
numerical-->carat,x,y,z,depth,table
ordinalencoding-->cut,color,clarity,

In [10]:
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler

In [12]:
data['cut'].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [13]:
data['color'].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [14]:
data['clarity'].unique()

array(['SI2', 'SI1', 'VS1', 'VS2', 'VVS2', 'VVS1', 'I1', 'IF'],
      dtype=object)

In [15]:
cu=['Fair','Good','Very Good','Premium', 'Ideal']
co=['J','I','H','G','F','E','D']
cl=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [16]:
oe=OrdinalEncoder(categories=[cu,co,cl]).set_output(transform='pandas')
oe

OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
                           ['J', 'I', 'H', 'G', 'F', 'E', 'D'],
                           ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1',
                            'IF']])

In [21]:
minmax=MinMaxScaler().set_output(transform='pandas')
minmax

MinMaxScaler()

In [19]:
oe.fit_transform(x_train[['cut','color','clarity']])

,cut,color,clarity
35965,1.0,5.0,5.0
52281,4.0,0.0,2.0
6957,3.0,0.0,3.0
9163,4.0,4.0,1.0
50598,4.0,4.0,4.0
...,...,...,...
11284,2.0,1.0,3.0
44732,4.0,6.0,4.0
38158,2.0,4.0,7.0
860,3.0,0.0,2.0


In [22]:
minmax.fit_transform(x_train[['carat','x','y','z','depth','table']])

,carat,x,y,z,depth,table
35965,0.010395,0.367784,0.067402,0.080818,0.608333,0.288462
52281,0.133056,0.562384,0.103056,0.117610,0.522222,0.250000
6957,0.176715,0.610801,0.110526,0.125472,0.502778,0.288462
9163,0.170478,0.608007,0.110357,0.124214,0.491667,0.250000
50598,0.085239,0.505587,0.092869,0.105975,0.522222,0.269231
...,...,...,...,...,...,...
11284,0.176715,0.603352,0.110526,0.127358,0.538889,0.307692
44732,0.056133,0.468343,0.085059,0.096226,0.500000,0.230769
38158,0.027027,0.418063,0.075722,0.084906,0.480556,0.288462
860,0.145530,0.570764,0.102377,0.120126,0.550000,0.307692


In [23]:
## creating a columtransformer for encoding and scaling at a time

In [24]:
from sklearn.compose import ColumnTransformer

In [27]:
ct=ColumnTransformer([('encoding',oe,['cut','color','clarity']),
                      ('minmax',minmax,['carat','x','y','z','depth','table'])],
                     verbose_feature_names_out=False).set_output(transform='pandas')
ct

ColumnTransformer(transformers=[('encoding',
                                 OrdinalEncoder(categories=[['Fair', 'Good',
                                                             'Very Good',
                                                             'Premium',
                                                             'Ideal'],
                                                            ['J', 'I', 'H', 'G',
                                                             'F', 'E', 'D'],
                                                            ['I1', 'SI2', 'SI1',
                                                             'VS2', 'VS1',
                                                             'VVS2', 'VVS1',
                                                             'IF']]),
                                 ['cut', 'color', 'clarity']),
                                ('minmax', MinMaxScaler(),
                                 ['carat', 'x', 'y', 'z', 'depth', 'table'])],
                  verbose_feature_names_out=False)

In [30]:
x_train_transformed=ct.fit_transform(x_train)
x_train_transformed

,cut,color,clarity,carat,x,y,z,depth,table
35965,1.0,5.0,5.0,0.010395,0.367784,0.067402,0.080818,0.608333,0.288462
52281,4.0,0.0,2.0,0.133056,0.562384,0.103056,0.117610,0.522222,0.250000
6957,3.0,0.0,3.0,0.176715,0.610801,0.110526,0.125472,0.502778,0.288462
9163,4.0,4.0,1.0,0.170478,0.608007,0.110357,0.124214,0.491667,0.250000
50598,4.0,4.0,4.0,0.085239,0.505587,0.092869,0.105975,0.522222,0.269231
...,...,...,...,...,...,...,...,...,...
11284,2.0,1.0,3.0,0.176715,0.603352,0.110526,0.127358,0.538889,0.307692
44732,4.0,6.0,4.0,0.056133,0.468343,0.085059,0.096226,0.500000,0.230769
38158,2.0,4.0,7.0,0.027027,0.418063,0.075722,0.084906,0.480556,0.288462
860,3.0,0.0,2.0,0.145530,0.570764,0.102377,0.120126,0.550000,0.307692


## knn manual model building

In [34]:
x_test_transformed=ct.transform(x_test)
x_test_transformed.head()

,cut,color,clarity,carat,x,y,z,depth,table
1388,4.0,3.0,6.0,0.008316,0.369646,0.067912,0.077673,0.530556,0.250000
50052,2.0,4.0,5.0,0.079002,0.506518,0.092020,0.102516,0.472222,0.269231
41645,4.0,5.0,5.0,0.041580,0.443203,0.080475,0.092767,0.530556,0.230769
42377,3.0,5.0,5.0,0.047817,0.458101,0.083022,0.093711,0.494444,0.269231
17244,4.0,5.0,1.0,0.280665,0.692737,0.125127,0.144969,0.536111,0.230769


In [102]:
query=x_test_transformed.sample(n=1,random_state=41)
query

,cut,color,clarity,carat,x,y,z,depth,table
42217,4.0,3.0,7.0,0.045738,0.445996,0.081834,0.094654,0.544444,0.192308


In [35]:
## import eculidean_distances

In [95]:
from sklearn.metrics.pairwise import euclidean_distances

In [103]:
dist=euclidean_distances(x_train_transformed,query)
dist

array([[4.12566278],
       [5.83317838],
       [5.10461264],
       ...,
       [2.23932805],
       [5.91945442],
       [5.20321322]])

In [47]:
import numpy as np

In [97]:
np.sort(dist,axis=0)

array([[0.00000000e+00],
       [2.83935346e-03],
       [2.83935346e-03],
       ...,
       [7.84639786e+00],
       [7.87994548e+00],
       [7.89598328e+00]])

In [98]:
sort=np.argsort(dist,axis=0)
sort

array([[38065],
       [ 5864],
       [37809],
       ...,
       [ 3288],
       [18043],
       [ 9125]])

In [99]:
sort_sim=sort[:5]
sort_sim

array([[38065],
       [ 5864],
       [37809],
       [24582],
       [ 3740]])

In [63]:
y_train.shape

(40455,)

In [58]:
y_train

35965     476
52281    2493
6957     4145
9163     4541
50598    2283
         ... 
11284    4975
44732    1617
38158    1014
860      2871
15795    6320
Name: price, Length: 40455, dtype: int64

In [100]:
y_train.values[sort_sim].mean()

np.float64(880.4)

## now with  model building

In [68]:
from sklearn.neighbors import KNeighborsRegressor

In [85]:
knn=KNeighborsRegressor(n_neighbors=5)
knn

KNeighborsRegressor()

In [86]:
knn.fit(x_train_transformed,y_train)

KNeighborsRegressor()

In [87]:
knn.score(x_train_transformed,y_train)

0.9755567437592142

In [ ]:
## comparision of knn model and manual knn values

In [88]:
knn.kneighbors(query),np.sort(dist,axis=0),sort

((array([[0.00016978, 0.00283935, 0.00375906, 0.0056356 , 0.006247  ]]),
  array([[ 8183, 30159, 25511, 21283, 22614]])),
 array([[1.69779183e-04],
        [2.83935346e-03],
        [3.75906126e-03],
        ...,
        [8.15944494e+00],
        [8.19192504e+00],
        [8.20742129e+00]]),
 array([[ 8183],
        [30159],
        [25511],
        ...,
        [ 3288],
        [18043],
        [ 9125]]))

In [ ]:
## both processes values  are same 

## evaluate

In [89]:
y_pred=knn.predict(x_test_transformed)
y_pred

array([ 586.4, 2163.8, 1083.4, ..., 8743.4, 4548.8, 4145.6])

In [76]:
y_test

1388       559
50052     2201
41645     1238
42377     1304
17244     6901
         ...  
36825      957
24803    13187
20999     9215
8471      4416
4204      3564
Name: price, Length: 13485, dtype: int64

In [90]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R2 score:", r2)


R2 score: 0.9621350175579151


In [101]:
knn.predict(query)

array([880.4])